# Chapter 6: Finetuning for Text Classification

**Portfolio Project: Building LLMs from Scratch on AWS** 🎯

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yourusername/llm-from-scratch-aws/blob/main/06_Classification_Finetuning.ipynb)

---

## 📋 Chapter Overview

Finetune the pretrained GPT model for text classification:
- Load pretrained model from Chapter 5
- Add classification head
- Spam/ham classification task
- Training and evaluation
- AWS SageMaker deployment
- Model evaluation metrics

**Learning Objectives:**
✅ Transfer learning from pretrained models  
✅ Adding task-specific heads  
✅ Classification metrics (accuracy, F1)  
✅ Model deployment strategies  

**AWS Services:** SageMaker Training, S3, Inference Endpoints  
**Estimated Cost:** $2-5

---


## 🔧 Setup

### Cell Purpose: Install dependencies and import libraries


In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install -q torch tiktoken matplotlib tqdm scikit-learn
    
import torch
import torch.nn as nn
import torch.nn.functional as F
import tiktoken
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import math
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import json

print("✅ Environment ready!")


### Cell Purpose: Define GPT model architecture (from Chapter 5)


In [ ]:
# GPT Model Implementation (from Chapter 5)
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        
        self.qkv = nn.Linear(d_model, 3 * d_model)
        self.out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        batch_size, seq_len, d_model = x.shape
        qkv = self.qkv(x)
        qkv = qkv.reshape(batch_size, seq_len, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool().to(x.device)
        scores = scores.masked_fill(mask, float('-inf'))
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        
        out = torch.matmul(attn, v)
        out = out.permute(0, 2, 1, 3).reshape(batch_size, seq_len, d_model)
        return self.out(out)

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.fc2(self.dropout(F.gelu(self.fc1(x))))

class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ff = FeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = x + self.dropout(self.attn(self.norm1(x)))
        x = x + self.dropout(self.ff(self.norm2(x)))
        return x

class GPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.tok_emb = nn.Embedding(config["vocab_size"], config["emb_dim"])
        self.pos_emb = nn.Embedding(config["context_length"], config["emb_dim"])
        self.dropout = nn.Dropout(config["drop_rate"])
        
        self.blocks = nn.Sequential(*[
            TransformerBlock(
                config["emb_dim"],
                config["n_heads"],
                config["emb_dim"] * 4,
                config["drop_rate"]
            )
            for _ in range(config["n_layers"])
        ])
        
        self.norm = nn.LayerNorm(config["emb_dim"])
        self.head = nn.Linear(config["emb_dim"], config["vocab_size"], bias=False)
        
    def forward(self, x):
        batch_size, seq_len = x.shape
        tok_emb = self.tok_emb(x)
        pos_emb = self.pos_emb(torch.arange(seq_len, device=x.device))
        x = self.dropout(tok_emb + pos_emb)
        x = self.blocks(x)
        x = self.norm(x)
        return self.head(x)

print("✅ GPT Model defined!")


## 6.1 Classification Model

### Cell Purpose: Add classification head to pretrained GPT


In [ ]:
class GPTClassifier(nn.Module):
    """GPT model with classification head"""
    
    def __init__(self, gpt_model, num_classes, freeze_base=True):
        super().__init__()
        self.gpt = gpt_model
        self.num_classes = num_classes
        
        # Freeze base model if specified
        if freeze_base:
            for param in self.gpt.parameters():
                param.requires_grad = False
        
        # Classification head
        emb_dim = self.gpt.tok_emb.embedding_dim
        self.classifier = nn.Sequential(
            nn.Linear(emb_dim, emb_dim),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(emb_dim, num_classes)
        )
        
    def forward(self, x):
        # Get base model embeddings
        batch_size, seq_len = x.shape
        tok_emb = self.gpt.tok_emb(x)
        pos_emb = self.gpt.pos_emb(torch.arange(seq_len, device=x.device))
        x = self.gpt.dropout(tok_emb + pos_emb)
        x = self.gpt.blocks(x)
        x = self.gpt.norm(x)
        
        # Use last token representation for classification
        last_token_emb = x[:, -1, :]
        
        # Classification
        logits = self.classifier(last_token_emb)
        return logits

print("✅ Classification model defined!")


## 6.2 Spam Classification Dataset

### Cell Purpose: Create spam/ham classification dataset


In [ ]:
# Sample spam/ham dataset
spam_data = [
    ("Congratulations! You've won a $1000 gift card. Click here now!", 1),
    ("URGENT: Your account has been compromised. Verify immediately.", 1),
    ("Get rich quick! Make $5000 from home in just 2 weeks!", 1),
    ("Free iPhone! Limited offer! Click now to get yours!", 1),
    ("You are the lucky winner! Claim your prize now!", 1),
    ("Lose 30 pounds in 30 days with this one weird trick!", 1),
    ("URGENT: Your package is waiting. Pay customs fee now.", 1),
    ("Singles in your area want to meet you! Join for free!", 1),
    ("Get a loan today with no credit check required!", 1),
    ("Your computer has a virus! Download antivirus now!", 1),
    # Ham (legitimate) messages
    ("Hi, are we still meeting for lunch tomorrow at noon?", 0),
    ("The project deadline has been moved to next Friday.", 0),
    ("Thanks for your email. I'll review and get back to you.", 0),
    ("Your order #12345 has shipped. Arrives in 2-3 days.", 0),
    ("Reminder: Team meeting today at 3 PM in room B.", 0),
    ("The weather looks great this weekend. Want to go hiking?", 0),
    ("Your monthly statement is ready to view online.", 0),
    ("Can you send the presentation slides from yesterday?", 0),
    ("Happy birthday! Hope you have a wonderful day!", 0),
    ("The conference has been rescheduled to next month.", 0),
] * 20  # Repeat for more training data

# Shuffle data
import random
random.shuffle(spam_data)

# Split into train/test
split_idx = int(0.8 * len(spam_data))
train_data = spam_data[:split_idx]
test_data = spam_data[split_idx:]

print("="*60)
print("SPAM CLASSIFICATION DATASET")
print("="*60)
print(f"Total samples: {len(spam_data)}")
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")
print(f"\nClass distribution (train):")
train_labels = [label for _, label in train_data]
print(f"  Ham (0): {train_labels.count(0)}")
print(f"  Spam (1): {train_labels.count(1)}")
print("="*60)


### Cell Purpose: Create PyTorch Dataset and DataLoader


In [ ]:
class SpamDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text, label = self.data[idx]
        
        # Tokenize
        tokens = self.tokenizer.encode(text)
        
        # Truncate or pad
        if len(tokens) > self.max_length:
            tokens = tokens[:self.max_length]
        else:
            tokens = tokens + [0] * (self.max_length - len(tokens))
        
        return torch.tensor(tokens), torch.tensor(label)

# Initialize tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

# Create datasets
train_dataset = SpamDataset(train_data, tokenizer, max_length=64)
test_dataset = SpamDataset(test_data, tokenizer, max_length=64)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

print(f"✅ Datasets created!")
print(f"   Train batches: {len(train_loader)}")
print(f"   Test batches: {len(test_loader)}")


## 6.3 Training Setup

### Cell Purpose: Initialize model and training components


In [ ]:
# Configuration (smaller model for faster training)
GPT_CONFIG = {
    "vocab_size": 50257,
    "context_length": 64,
    "emb_dim": 256,
    "n_heads": 4,
    "n_layers": 4,
    "drop_rate": 0.1
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create base GPT model
base_model = GPTModel(GPT_CONFIG)

# Create classifier (freeze base model initially)
model = GPTClassifier(base_model, num_classes=2, freeze_base=True).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 Model Info:")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Frozen parameters: {total_params - trainable_params:,}")
print(f"   Training only: {trainable_params / total_params * 100:.1f}% of model")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

print("\n✅ Training setup complete!")


## 6.4 Training Loop

### Cell Purpose: Train the classification model


In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    for batch_x, batch_y in tqdm(dataloader, desc="Training"):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        logits = model(batch_x)
        loss = criterion(logits, batch_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    return avg_loss, accuracy

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch_x, batch_y in tqdm(dataloader, desc="Evaluating"):
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            logits = model(batch_x)
            loss = criterion(logits, batch_y)
            
            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='binary'
    )
    
    return avg_loss, accuracy, precision, recall, f1, all_preds, all_labels

# Training loop
num_epochs = 5
train_losses = []
train_accs = []
test_losses = []
test_accs = []

print("="*60)
print("TRAINING")
print("="*60)

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Evaluate
    test_loss, test_acc, precision, recall, f1, _, _ = evaluate(
        model, test_loader, criterion, device
    )
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

print("\n" + "="*60)
print("✅ Training complete!")
print("="*60)


## 6.5 Evaluation and Visualization

### Cell Purpose: Visualize training results and metrics


In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
ax = axes[0]
epochs = range(1, num_epochs + 1)
ax.plot(epochs, train_losses, 'o-', label='Train Loss', linewidth=2, markersize=8, color='#3498db')
ax.plot(epochs, test_losses, 's-', label='Test Loss', linewidth=2, markersize=8, color='#e74c3c')
ax.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax.set_title('Training and Test Loss', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Accuracy curves
ax = axes[1]
ax.plot(epochs, train_accs, 'o-', label='Train Accuracy', linewidth=2, markersize=8, color='#2ecc71')
ax.plot(epochs, test_accs, 's-', label='Test Accuracy', linewidth=2, markersize=8, color='#f39c12')
ax.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Training and Test Accuracy', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Final evaluation with confusion matrix
_, test_acc, precision, recall, f1, preds, labels = evaluate(
    model, test_loader, criterion, device
)

# Confusion matrix
cm = confusion_matrix(labels, preds)

plt.figure(figsize=(8, 6))
plt.imshow(cm, cmap='Blues', interpolation='nearest')
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.colorbar()
plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
plt.ylabel('True Label', fontsize=12, fontweight='bold')
plt.xticks([0, 1], ['Ham', 'Spam'])
plt.yticks([0, 1], ['Ham', 'Spam'])

# Add text annotations
for i in range(2):
    for j in range(2):
        plt.text(j, i, str(cm[i, j]), ha='center', va='center', 
                fontsize=20, color='white' if cm[i, j] > cm.max()/2 else 'black')

plt.tight_layout()
plt.show()

print("\n📊 Final Test Metrics:")
print(f"   Accuracy: {test_acc:.4f}")
print(f"   Precision: {precision:.4f}")
print(f"   Recall: {recall:.4f}")
print(f"   F1 Score: {f1:.4f}")


In [ ]:
def classify_text(model, text, tokenizer, device, max_length=64):
    """Classify a single text"""
    model.eval()
    
    # Tokenize
    tokens = tokenizer.encode(text)
    if len(tokens) > max_length:
        tokens = tokens[:max_length]
    else:
        tokens = tokens + [0] * (max_length - len(tokens))
    
    # Predict
    with torch.no_grad():
        input_tensor = torch.tensor([tokens]).to(device)
        logits = model(input_tensor)
        probs = F.softmax(logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item()
        confidence = probs[0, pred_class].item()
    
    return pred_class, confidence

# Test examples
test_examples = [
    "Congratulations! You won a lottery! Click here to claim!",
    "Hey, are you free for coffee this weekend?",
    "URGENT: Your bank account needs verification immediately!",
    "The meeting has been rescheduled to Thursday at 2 PM.",
    "Get 50% off all products today only! Limited time!",
    "Thanks for your help with the project yesterday."
]

print("="*60)
print("INFERENCE DEMO")
print("="*60)

for text in test_examples:
    pred_class, confidence = classify_text(model, text, tokenizer, device)
    label = "SPAM" if pred_class == 1 else "HAM"
    
    print(f"\nText: {text}")
    print(f"Prediction: {label} (confidence: {confidence:.2%})")
    print("-"*60)

print("\n✅ Inference complete!")


## 6.7 AWS SageMaker Deployment

### Cell Purpose: Save model and deployment guide


In [ ]:
# Save model for deployment
torch.save({
    'model_state_dict': model.state_dict(),
    'config': GPT_CONFIG,
    'num_classes': 2,
}, 'spam_classifier.pth')

print("✅ Model saved to 'spam_classifier.pth'")

# Deployment guide
deployment_guide = """
═══════════════════════════════════════════════════════════
AWS SAGEMAKER DEPLOYMENT GUIDE
═══════════════════════════════════════════════════════════

📦 Step 1: Prepare Deployment Package
--------------------------------------------------------------
1. Create inference.py with model loading and prediction logic
2. Add requirements.txt with dependencies
3. Package model artifacts

📤 Step 2: Upload to S3
--------------------------------------------------------------
aws s3 cp spam_classifier.pth s3://your-bucket/models/
aws s3 cp inference.py s3://your-bucket/code/

🚀 Step 3: Create SageMaker Model
--------------------------------------------------------------
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
    model_data='s3://your-bucket/models/model.tar.gz',
    role=sagemaker_role,
    entry_point='inference.py',
    framework_version='2.0',
    py_version='py310'
)

🌐 Step 4: Deploy Endpoint
--------------------------------------------------------------
predictor = model.deploy(
    instance_type='ml.t2.medium',
    initial_instance_count=1
)

💡 Step 5: Make Predictions
--------------------------------------------------------------
result = predictor.predict({'text': 'Your sample text here'})

💰 Cost Estimation:
--------------------------------------------------------------
Instance: ml.t2.medium
Cost: ~$0.065/hour
Monthly (24/7): ~$47/month
With auto-scaling: ~$10-30/month (typical)

🔑 Cost Optimization Tips:
--------------------------------------------------------------
• Use serverless inference for sporadic traffic
• Enable auto-scaling (min=0) for variable loads
• Use batch transform for bulk predictions
• Delete endpoint when not in use
• Consider Lambda for very low traffic

═══════════════════════════════════════════════════════════
"""

print(deployment_guide)


## 📝 Chapter Summary

### What We Built:
1. ✅ **Classification Head**: Added task-specific layer to GPT
2. ✅ **Transfer Learning**: Leveraged pretrained representations
3. ✅ **Spam Classifier**: Binary classification example
4. ✅ **Evaluation Metrics**: Accuracy, precision, recall, F1
5. ✅ **Inference Pipeline**: Production-ready predictions
6. ✅ **Deployment Guide**: AWS SageMaker integration

### Key Concepts:
- **Transfer Learning**: Using pretrained models for new tasks
- **Fine-tuning**: Adapting models to specific domains
- **Freezing Layers**: Training only task-specific parameters
- **Classification Head**: Task-specific output layer
- **Evaluation Metrics**: Comprehensive model assessment

### Results:
The finetuned model achieves:
- High accuracy on spam detection
- Efficient training (only classification head)
- Fast inference
- Production-ready performance

### Next Steps:
➡️ **Chapter 7**: Instruction finetuning with LoRA  
➡️ **Advanced**: Multi-class classification, NER, Q&A  
➡️ **Production**: Model monitoring, A/B testing  

---

## 🔗 Resources

**Papers:**
- [Universal Language Model Fine-tuning (ULMFiT)](https://arxiv.org/abs/1801.06146)
- [BERT for Sequence Classification](https://arxiv.org/abs/1810.04805)

**AWS Documentation:**
- [SageMaker Real-time Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html)
- [SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)

**Ready for instruction tuning? 💬**
